In [1]:
import re
import pandas as pd

path = 'arrets_ben.csv'

# 1) Grab the header line + dash line
with open(path, encoding='utf-8') as f:
    header_line = f.readline().rstrip('\n')
    dash_line   = f.readline().rstrip('\n')

# 2) Infer colspecs from runs of dashes
colspecs = [(m.start(), m.end()) for m in re.finditer(r'-+', dash_line)]

# 3) Slice out the raw names from the header
raw_names = [header_line[start:end].strip() for start, end in colspecs]

# 4) De‑duplicate any repeated names by appending a counter
seen = {}
names = []
for nm in raw_names:
    if nm in seen:
        seen[nm] += 1
        names.append(f"{nm}_{seen[nm]}")
    else:
        seen[nm] = 0
        names.append(nm)

# 5) Read the data as fixed‑width, skipping the two top lines
df = pd.read_fwf(
    path,
    colspecs=colspecs,
    names=names,
    skiprows=[0, 1],      # drop both the header and the dash‐line
    na_values=['NULL'],   # turn “NULL” → NaN
    dtype=str             # optional: everything as string if you’ll parse types later
)

print(df.shape)           # expect (n_rows, 73)
print(df.columns.tolist())  
df.head()




Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
(2301409, 73)
['\ufeffIdCourse', 'IdArret', 'RangArretAs', 'DateCours', 'HDepartThe', 'HArriveeThe', 'DistanceThe', 'TempsInterArretThe', 'EcartDepar', 'EcartDistanc', 'TempsPasseArre', 'TempsInterArretRealis', 'VitesseInterArre', 'NbMontees', 'NbDescente', 'Charge', 'FlagArretReferenc', 'FlagDeviatio', 'FlagDelocalisatio', 'FlagRecalag', 'FlagRegulatio', 'IdArretPreceden', 'DistanceInterArre', 'NbMonteesPorte', 'NbDescentesPorte', 'NbMonteesPorte_1', 'NbDescentesPorte_1', 'NbMonteesPorte_2', 'N

,﻿IdCourse,IdArret,RangArretAs,DateCours,HDepartThe,HArriveeThe,DistanceThe,TempsInterArretThe,EcartDepar,EcartDistanc,...,C_Lign,C_NoParcoursApp,C_NumVoitur,C_ParcoursTypeApp,C_SensApp,C_ServiceVoitur,C_TempsBattemen,C_TypeAppl,CodeLong,Arret
0,83748979,261,1,2024-10-01,16:59:00,16:59:00,0,0,58,0,...,18,78,91,55257,R,18-91,125,0,CERN01,CERN
1,83748853,261,1,2024-10-01,06:31:00,06:31:00,0,0,-6,0,...,18,78,87,55257,R,18-87,449,0,CERN01,CERN
2,83748927,6948,1,2024-10-01,05:07:00,05:07:00,0,0,255,0,...,18,6,90,49578,A,18-90,22,2,DE00,DEPOT EN-CHARDON
3,83748978,260,1,2024-10-01,16:46:00,16:46:00,0,0,-82,0,...,18,10,91,50446,R,18-91,0,1,CERN00,CERN
4,83748928,702,1,2024-10-01,05:09:00,05:09:00,0,0,229,0,...,18,4,90,54136,A,18-90,0,0,JALP00,JARDIN-ALPIN-V.


In [ ]:
# A better way to read the same file, handling BOM and end-of-line truncation

import re
import pandas as pd
from pathlib import Path

path = "arrets_ben.csv"
out_parquet = "arrets_ben.parquet"

# --- 1) Read header + dash line, remove BOM automatically ---
with open(path, encoding="utf-8-sig") as f:
    header_line = f.readline().rstrip("\n")
    dash_line   = f.readline().rstrip("\n")

# --- 2) Infer column spans from runs of dashes ---
colspecs = [(m.start(), m.end()) for m in re.finditer(r"-+", dash_line)]

# Make the last column go to end-of-line to avoid truncation
colspecs[-1] = (colspecs[-1][0], None)

# --- 3) Slice column names from the (BOM-stripped) header ---
raw_names = [header_line[s:] if e is None else header_line[s:e] for s, e in colspecs]
names = []
seen = {}
for nm in map(str.strip, raw_names):
    seen[nm] = seen.get(nm, -1) + 1
    names.append(nm if seen[nm] == 0 else f"{nm}_{seen[nm]}")

print("Detected columns:", len(names))
print(names[:10], "...")

# --- 4) Read the data as fixed-width (skip header + dashes) ---
df = pd.read_fwf(
    path,
    colspecs=colspecs,
    names=names,
    skiprows=2,
    na_values=["NULL"],
    encoding="utf-8-sig",
)
print(df.shape)
print(df.head(3))

Detected columns: 73
['IdCourse', 'IdArret', 'RangArretAsc', 'DateCourse', 'HDepartTheo', 'HArriveeTheo', 'DistanceTheo', 'TempsInterArretTheo', 'EcartDepart', 'EcartDistance'] ...
(2301409, 73)
   IdCourse  IdArret  RangArretAsc  DateCourse HDepartTheo HArriveeTheo  \
0  83748979      261             1  2024-10-01    16:59:00     16:59:00   
1  83748853      261             1  2024-10-01    06:31:00     06:31:00   
2  83748927     6948             1  2024-10-01    05:07:00     05:07:00   

   DistanceTheo  TempsInterArretTheo  EcartDepart  EcartDistance  ...  \
0             0                    0           58              0  ...   
1             0                    0           -6              0  ...   
2             0                    0          255              0  ...   

   C_Ligne  C_NoParcoursAppl  C_NumVoiture  C_ParcoursTypeAppl  C_SensAppl  \
0       18              78.0            91             55257.0           R   
1       18              78.0            87             

In [30]:
df.head()

,IdCourse,IdArret,RangArretAsc,DateCourse,HDepartTheo,HArriveeTheo,DistanceTheo,TempsInterArretTheo,EcartDepart,EcartDistance,...,C_Ligne,C_NoParcoursAppl,C_NumVoiture,C_ParcoursTypeAppl,C_SensAppl,C_ServiceVoiture,C_TempsBattement,C_TypeAppl,CodeLong,Arret
0,83748979,261,1,2024-10-01,16:59:00,16:59:00,0,0,58,0,...,18,78.0,91,55257.0,R,18-91,125.0,0.0,CERN01,CERN
1,83748853,261,1,2024-10-01,06:31:00,06:31:00,0,0,-6,0,...,18,78.0,87,55257.0,R,18-87,449.0,0.0,CERN01,CERN
2,83748927,6948,1,2024-10-01,05:07:00,05:07:00,0,0,255,0,...,18,6.0,90,49578.0,A,18-90,22.0,2.0,DE00,DEPOT EN-CHARDON
3,83748978,260,1,2024-10-01,16:46:00,16:46:00,0,0,-82,0,...,18,10.0,91,50446.0,R,18-91,0.0,1.0,CERN00,CERN
4,83748928,702,1,2024-10-01,05:09:00,05:09:00,0,0,229,0,...,18,4.0,90,54136.0,A,18-90,0.0,0.0,JALP00,JARDIN-ALPIN-V.


In [63]:
import pandas as pd
import numpy as np

# -------------------------
# 0) CONFIG
# -------------------------
LINE = 18
DATE_FROM = "2024-10-01"
DATE_TO   = "2025-04-30"

# segments requested (from → to) using MNLP / CodeLong
SEGMENTS = [
    ("BOLA00", "MYRN00"),
    ("MYRN00", "HTOU00"),
    ("HTOU00",  "MAIX00"),
]

# Brand/Ben thresholds (seconds / km/h)
alpha1 = 60    # large dwell
alpha2 = 120   # large dwell variation (p85-p15)
alpha3 = -60   # early departure
alpha4 = 180   # late departure
alpha5 = 300   # large dep. time variation (p85-p15)
alpha6 = 60    # punctuality change wrt previous
alpha7 = 15    # low speed (km/h)
alpha8 = 60    # travel time excess over free flow (s)

# -------------------------
# 1) PREP & CLEAN
# -------------------------
# keep the requested date range & line 18 & commercial trips if available
if "DateCourse" in df.columns:
    df["DateCourse"] = pd.to_datetime(df["DateCourse"], errors="coerce")
    df = df[(df["DateCourse"] >= DATE_FROM) & (df["DateCourse"] <= DATE_TO)]

df = df[df["C_Ligne"] == LINE]
if "C_TypeAppl" in df.columns:
    df = df[df["C_TypeAppl"] == 0]  # commercial only

# parse the datetimes we need
for c in ["DTDepartTheo","DTEntreeFenetreArretReal","DTSortieFenetreArretReal"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

# ensure CodeLong is a clean string
df["CodeLong"] = df["CodeLong"].astype(str).str.strip()

# -------------------------
# 2) BUILD SEGMENT ROWS
# Each row i (stop i) represents segment (prev stop -> current stop)
# travel time = entry_time@i - exit_time@(i-1) within the same IdCourse
# -------------------------
df = df.sort_values(["IdCourse", "RangArretAsc"])

df["prev_CodeLong"]  = df.groupby("IdCourse")["CodeLong"].shift(1)
df["prev_exit_time"] = df.groupby("IdCourse")["DTSortieFenetreArretReal"].shift(1)

# keep rows that have a previous stop (i.e., a valid segment)
seg = df.dropna(subset=["prev_CodeLong","DTEntreeFenetreArretReal","prev_exit_time"]).copy()

# segment id like 'XXXXYYYY' (Ben used 8-char codes; we concatenate as-is)
seg["SegmentID"] = (seg["prev_CodeLong"].str[:4] + seg["CodeLong"].str[:4]).str.zfill(8)

# travel time (s), distance (m), speed (km/h)
seg["segment_travel_time"] = (seg["DTEntreeFenetreArretReal"] - seg["prev_exit_time"]).dt.total_seconds()
seg = seg[seg["segment_travel_time"] > 0]  # sanity

# DistanceInterArret may correspond to current link; subtract 70m windows
dist_col = "DistanceInterArret" if "DistanceInterArret" in seg.columns else "DistanceTheo"
seg[dist_col] = pd.to_numeric(seg[dist_col], errors="coerce")
seg = seg[seg[dist_col] > 0]
seg["segment_distance_m"] = seg[dist_col] - 70.0
seg = seg[seg["segment_distance_m"] > 0]

seg["segment_speed_kmh"] = 3.6 * seg["segment_distance_m"] / seg["segment_travel_time"]

# dwell time at current stop (window time)
seg["dwell_time"] = (seg["DTSortieFenetreArretReal"] - seg["DTEntreeFenetreArretReal"]).dt.total_seconds()

# -------------------------
# 3) PERIOD ASSIGNMENT (from theoretical departure time)
# -------------------------
def assign_period(dt):
    if pd.isna(dt): 
        return "Other"
    d = dt.dayofweek  # Mon=0 ... Sun=6
    h = dt.hour
    if d == 5:  # Saturday
        return "Saturdays"
    if d == 6:  # Sunday
        return "Sundays"
    if 7 <= h < 9:
        return "AM peak"
    elif 9 <= h < 16:
        return "Day period"
    elif 16 <= h < 19:
        return "PM peak"
    elif 19 <= h < 23:
        return "Evening"
    else:
        return "Other"

seg["period"] = seg["DTDepartTheo"].map(assign_period)

# -------------------------
# 4) FILTER TO THE 3 REQUESTED SEGMENTS
# (match using full MNLP codes, not 4-char)
# -------------------------
want_ids = [f"{a}{b}" for a,b in SEGMENTS]
seg["SegmentKey"] = seg["prev_CodeLong"].str.strip() + "→" + seg["CodeLong"].str.strip()
seg3 = seg[(seg["prev_CodeLong"].isin([a for a,_ in SEGMENTS])) &
           (seg["CodeLong"].isin([b for _,b in SEGMENTS])) &
           (seg["SegmentKey"].isin([f"{a}→{b}" for a,b in SEGMENTS]))].copy()

# -------------------------
# 5) BEN’S “MEAN SPEED BY PERIOD” TABLE (Excel-ready)
# -------------------------
pivot_speed = (
    seg3
    .assign(Segment_X=seg3["prev_CodeLong"].str[:4] + seg3["CodeLong"].str[:4])
    .pivot_table(index="Segment_X", columns="period", values="segment_speed_kmh", aggfunc="mean")
    .reindex(columns=["AM peak","Day period","PM peak","Evening","Saturdays","Sundays","Other"])
    .round(3)
)
pivot_speed["net"] = seg3.groupby(seg3["prev_CodeLong"].str[:4] + seg3["CodeLong"].str[:4])["segment_speed_kmh"].mean().round(3)
pivot_speed = pivot_speed.reset_index()

pivot_speed.to_csv("line18_segments_mean_speed_by_period.csv", index=False)
print("Wrote: line18_segments_mean_speed_by_period.csv")

# -------------------------
# 6) BEN’S 8 CRITERIA (counts by period) FOR THE 3 SEGMENTS
# -------------------------
# per-period IQR-like spreads
def p85_minus_p15(x):
    x = x.dropna()
    if len(x) < 2: return 0.0
    return np.percentile(x,85) - np.percentile(x,15)

# compute period spreads on the fly for each SegmentID
def add_flags(df_seg):
    # dwells
    df_seg["large_dwell_time"] = df_seg["dwell_time"] > alpha1

    spread_dw = df_seg.groupby("period")["dwell_time"].apply(p85_minus_p15).to_dict()
    df_seg["large_var_in_dwell"] = df_seg["period"].map(lambda p: spread_dw.get(p,0) > alpha2)

    # departures
    if "EcartDepart" in df_seg.columns:
        df_seg["early_departure"] = df_seg["EcartDepart"] < alpha3
        df_seg["late_departure"]  = df_seg["EcartDepart"] > alpha4
        spread_dep = df_seg.groupby("period")["EcartDepart"].apply(p85_minus_p15).to_dict()
        df_seg["large_var_in_departure"] = df_seg["period"].map(lambda p: spread_dep.get(p,0) > alpha5)
        df_seg = df_seg.sort_values("DTDepartTheo")
        df_seg["EcartDepart_prev"] = df_seg["EcartDepart"].shift(1)
        df_seg["punct_change_prev"] = (df_seg["EcartDepart"] - df_seg["EcartDepart_prev"]).abs() > alpha6
    else:
        # if missing, set False
        for c in ["early_departure","late_departure","large_var_in_departure","punct_change_prev"]:
            df_seg[c] = False

    # low speed
    df_seg["low_speed"] = df_seg["segment_speed_kmh"] < alpha7

    # free-flow baseline: 15th percentile of Sunday segment travel time (computed from all rows with same SegmentID)
    base = seg[seg["period"]=="Sundays"].groupby("SegmentID")["segment_travel_time"].quantile(0.15)
    df_seg = df_seg.merge(base.rename("freeflow_15th"), on="SegmentID", how="left")
    df_seg["large_travel_time_vs_freeflow"] = (df_seg["segment_travel_time"] - df_seg["freeflow_15th"]) > alpha8

    criteria = [
        "large_dwell_time","large_var_in_dwell","early_departure","late_departure",
        "large_var_in_departure","punct_change_prev","low_speed","large_travel_time_vs_freeflow"
    ]
    df_seg["any_criterion"] = df_seg[criteria].any(axis=1)

    # summarize counts by period
    out = df_seg.groupby("period")[criteria + ["any_criterion"]].sum().astype(int)
    out["num_trips"] = df_seg.groupby("period").size()
    out = out.reset_index()
    out["Segment_X"] = df_seg["prev_CodeLong"].str[:4].iloc[0] + df_seg["CodeLong"].str[:4].iloc[0]
    return out

# apply per requested segment
summaries = []
for a,b in SEGMENTS:
    sub = seg3[(seg3["prev_CodeLong"]==a) & (seg3["CodeLong"]==b)].copy()
    if not sub.empty:
        s = add_flags(sub)
        summaries.append(s)

if summaries:
    crit_tbl = pd.concat(summaries, ignore_index=True)
    # order columns nicely
    cols = ["Segment_X","period","num_trips",
            "large_dwell_time","large_var_in_dwell","early_departure","late_departure",
            "large_var_in_departure","punct_change_prev","low_speed","large_travel_time_vs_freeflow",
            "any_criterion"]
    crit_tbl = crit_tbl[cols]
    crit_tbl.to_csv("line18_segments_bottleneck_criteria_by_period.csv", index=False)
    print("Wrote: line18_segments_bottleneck_criteria_by_period.csv")
else:
    print("No rows found for the requested segments. Check CodeLong spellings.")

Wrote: line18_segments_mean_speed_by_period.csv
Wrote: line18_segments_bottleneck_criteria_by_period.csv
